In [1]:

import torch

In [2]:
torch.cuda.empty_cache()

In [ ]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
import torch 
from sklearn.model_selection import TimeSeriesSplit
from dash import Dash,dcc, html, Input, Output, callback



app = Dash(__name__)
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                'Pharmacy Forms',
                id="y"
        )
        ],style={"width":"33%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov"
        )
        ],style={"width":"33%","float":"right",'display':'inline-block'})
    ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            id="prop",style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ])
        ]
    )
@callback(
    Output("plotindic","figure"),
    Input("y","value"),
    Input("cov","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
     'name',
     'description',
     'metaKeywords',
     '_categories',
     '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
     '_title',
     '_slug',
     '_description',
     '_language',
     '_featuredCategory',
     'formID',
     'form_type',
     'source',
     'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
          df2=df.copy()
    elif cov=="Covid":
          df2=df.loc["01-07-2021":,]
    df2=df2.resample('3D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=122,model="add",extrapolate_trend="freq")
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                      subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=1000)
    return fig

if __name__ == '__main__':
	app.run(host='127.0.0.1', port='8050',debug=True)
    


In [ ]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
import torch 
from sklearn.model_selection import TimeSeriesSplit
from dash import Dash,dcc, html, Input, Output, callback



app = Dash(__name__)
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                'Pharmacy Forms',
                id="y"
        )
        ],style={"width":"33%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov"
        )
        ],style={"width":"33%","float":"right",'display':'inline-block'})
    ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="prop")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ])
        ]
    )
@callback(
    Output("plotindic","figure"),
    Input("y","value"),
    Input("cov","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
     'name',
     'description',
     'metaKeywords',
     '_categories',
     '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
     '_title',
     '_slug',
     '_description',
     '_language',
     '_featuredCategory',
     'formID',
     'form_type',
     'source',
     'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
          df2=df.copy()
    elif cov=="Covid":
          df2=df.loc["01-07-2021":,]
    df3=df2.copy()
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                      subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig


if __name__ == '__main__':
	app.run(host='127.0.0.1', port='8050',debug=True)
    


In [ ]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
import torch 
from sklearn.model_selection import TimeSeriesSplit
from dash import Dash,dcc, html, Input, Output, callback



app = Dash(__name__)
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                ['Pharmacy Forms',"Tax Forms"],
                id="y",
                multi=True,
                searchable=True
        )
        ],style={"width":"33%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov"
        )
        ],style={"width":"33%","float":"right",'display':'inline-block'})
    ]),
    html.Div(
        [
        html.Div(
            
            id="plotindic1",style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            id="plotindic2",style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic3")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic4")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic5")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic6")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic7")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic8")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic9")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic10")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
        ]
    )

@callback(
    Output("plotindic1","children"),
    Input("y","value"),
    Input("cov","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    if type(yaxis) is not list:
        formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
        formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
        formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
        formtemplates=formtemplates.rename({"_id":"id"},axis=1)
        formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
        formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
        df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
        formtemplates=df_melt._categories.apply(pd.Series) \
            .merge(formtemplates, right_index=True, left_index=True) \
            .drop(["_categories"], axis=1) \
            .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
            .drop("variable", axis=1) 
        formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
        formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"id":"_id"},axis=1)
        formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
        formcat=pd.concat([formtemplates2,formtemplates3])
        formcat=formcat.rename({"id":"_id"},axis=1)
        formcat=formcat.loc[:,[
        'name',
        'description',
        'metaKeywords',
        '_categories',
        '_id']]
        formandclone=formandclone.loc[:,[
        '_id',
        '_title',
        '_slug',
        '_description',
        '_language',
        '_featuredCategory',
        'formID',
        'form_type',
        'source',
        'date']]
        formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
        formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
        formclonecategnonnany.index=formclonecategnonnany.date
        formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
        formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
        formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
        formclonecategnonnany["week"]=formclonecategnonnany.index.week
        formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
        formclonecategnonnany["year"]=formclonecategnonnany.index.year
        formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
        formclonecategnonnany["month"]=formclonecategnonnany.index.month
        formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
        df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
        df.index=df.date.tolist()
        df.index=pd.to_datetime(df.index)
        df.index=df.index.to_period("D")
        df.index=df.index.to_timestamp()
        df.index=df.index.strftime('%d-%m-%Y')
        df["date"]=pd.to_datetime(df["date"])
        df=df.rename({"count(_id)":"y"},axis=1)
        df=df.rename({"date":"ds"},axis=1)
        df=df.loc[:,["y","ds"]]
        if cov=="Noncovid":
            df2=df.copy()
        elif cov=="Covid":
            df2=df.loc["01-07-2021":,]
        df2=df2.resample('7D',on="ds").sum()
        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
        season=season.fillna(0)
        season=season.sort_index()
        season.index=season.index.to_period("D")
        season.index=season.index.to_timestamp()
        season.index=season.index.strftime('%d-%m-%Y')
        season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
        fig=make_subplots(rows=4, cols=1,
                        subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
        fig.add_trace(
            go.Scatter(x=season.index,y=season.obs),row=1,col=1
            
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.trend),row=2,col=1
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
            
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.resid),row=4,col=1
            
        )


        fig.update_layout(height=1000, width=950)
        return [dcc.Graph(id="plotindic1",figure=fig)]
    elif type(yaxis) is list and len(yaxis)>=2:
        formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis[0]].copy()
        formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
        formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
        formtemplates=formtemplates.rename({"_id":"id"},axis=1)
        formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
        formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
        df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
        formtemplates=df_melt._categories.apply(pd.Series) \
            .merge(formtemplates, right_index=True, left_index=True) \
            .drop(["_categories"], axis=1) \
            .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
            .drop("variable", axis=1) 
        formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
        formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"id":"_id"},axis=1)
        formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
        formcat=pd.concat([formtemplates2,formtemplates3])
        formcat=formcat.rename({"id":"_id"},axis=1)
        formcat=formcat.loc[:,[
        'name',
        'description',
        'metaKeywords',
        '_categories',
        '_id']]
        formandclone=formandclone.loc[:,[
        '_id',
        '_title',
        '_slug',
        '_description',
        '_language',
        '_featuredCategory',
        'formID',
        'form_type',
        'source',
        'date']]
        formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
        formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
        formclonecategnonnany.index=formclonecategnonnany.date
        formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
        formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
        formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
        formclonecategnonnany["week"]=formclonecategnonnany.index.week
        formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
        formclonecategnonnany["year"]=formclonecategnonnany.index.year
        formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
        formclonecategnonnany["month"]=formclonecategnonnany.index.month
        formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
        df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
        df.index=df.date.tolist()
        df.index=pd.to_datetime(df.index)
        df.index=df.index.to_period("D")
        df.index=df.index.to_timestamp()
        df.index=df.index.strftime('%d-%m-%Y')
        df["date"]=pd.to_datetime(df["date"])
        df=df.rename({"count(_id)":"y"},axis=1)
        df=df.rename({"date":"ds"},axis=1)
        df=df.loc[:,["y","ds"]]
        if cov=="Noncovid":
            df2=df.copy()
        elif cov=="Covid":
            df2=df.loc["01-07-2021":,]
        df2=df2.resample('7D',on="ds").sum()
        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
        season=season.fillna(0)
        season=season.sort_index()
        season.index=season.index.to_period("D")
        season.index=season.index.to_timestamp()
        season.index=season.index.strftime('%d-%m-%Y')
        season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
        fig=make_subplots(rows=4, cols=1,
                        subplot_titles=(yaxis[0], "Trend", "Seasonal_yearly","resid"))
        fig.add_trace(
            go.Scatter(x=season.index,y=season.obs),row=1,col=1
            
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.trend),row=2,col=1
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
            
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.resid),row=4,col=1
            
        )


        fig.update_layout(height=1000, width=950)
        return [dcc.Graph(id="plotindic1",figure=fig)]
    
@callback(
    Output("plotindic2","children"),
    Input("y","value"),
    Input("cov","value")
)
def update_graph2(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    if type(yaxis) is  list and len(yaxis)==2:
        formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis[1]].copy()
        formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
        formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
        formtemplates=formtemplates.rename({"_id":"id"},axis=1)
        formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
        formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
        df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
        formtemplates=df_melt._categories.apply(pd.Series) \
            .merge(formtemplates, right_index=True, left_index=True) \
            .drop(["_categories"], axis=1) \
            .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
            .drop("variable", axis=1) 
        formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
        formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"id":"_id"},axis=1)
        formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
        formcat=pd.concat([formtemplates2,formtemplates3])
        formcat=formcat.rename({"id":"_id"},axis=1)
        formcat=formcat.loc[:,[
        'name',
        'description',
        'metaKeywords',
        '_categories',
        '_id']]
        formandclone=formandclone.loc[:,[
        '_id',
        '_title',
        '_slug',
        '_description',
        '_language',
        '_featuredCategory',
        'formID',
        'form_type',
        'source',
        'date']]
        formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
        formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
        formclonecategnonnany.index=formclonecategnonnany.date
        formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
        formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
        formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
        formclonecategnonnany["week"]=formclonecategnonnany.index.week
        formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
        formclonecategnonnany["year"]=formclonecategnonnany.index.year
        formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
        formclonecategnonnany["month"]=formclonecategnonnany.index.month
        formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
        df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
        df.index=df.date.tolist()
        df.index=pd.to_datetime(df.index)
        df.index=df.index.to_period("D")
        df.index=df.index.to_timestamp()
        df.index=df.index.strftime('%d-%m-%Y')
        df["date"]=pd.to_datetime(df["date"])
        df=df.rename({"count(_id)":"y"},axis=1)
        df=df.rename({"date":"ds"},axis=1)
        df=df.loc[:,["y","ds"]]
        if cov=="Noncovid":
            df2=df.copy()
        elif cov=="Covid":
            df2=df.loc["01-07-2021":,]
        df2=df2.resample('7D',on="ds").sum()
        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
        season=season.fillna(0)
        season=season.sort_index()
        season.index=season.index.to_period("D")
        season.index=season.index.to_timestamp()
        season.index=season.index.strftime('%d-%m-%Y')
        season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
        fig1=make_subplots(rows=4, cols=1,
                        subplot_titles=(yaxis[1], "Trend", "Seasonal_yearly","resid"))
        fig1.add_trace(
            go.Scatter(x=season.index,y=season.obs),row=1,col=1
            
        )
        fig1.add_trace(
            go.Scatter(x=season.index,y=season.trend),row=2,col=1
        )
        fig1.add_trace(
            go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
            
        )
        fig1.add_trace(
            go.Scatter(x=season.index,y=season.resid),row=4,col=1
            
        )

        fig1.update_layout(height=1000, width=950)
    return [dcc.Graph(id="plotindic2",figure=fig1)]
if __name__ == '__main__':
	app.run(host='127.0.0.1', port='8050',debug=True)
    


In [ ]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
import torch 
from sklearn.model_selection import TimeSeriesSplit
from dash import Dash,dcc, html, Input, Output, callback



app = Dash(__name__)
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                ['Pharmacy Forms',"Tax Forms"],
                id="y",
                multi=True,
                searchable=True
        )
        ],style={"width":"33%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov"
        )
        ],style={"width":"33%","float":"right",'display':'inline-block'})
    ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic1")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic2")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic3")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic4")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic5")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic6")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic7")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic8")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic9")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic10")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
        ]
    )

@callback(
    Output("plotindic1","figure"),
    Output("plotindic2","figure"),
    Input("y","value"),
    Input("cov","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    if type(yaxis) is not list:
        formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
        formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
        formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
        formtemplates=formtemplates.rename({"_id":"id"},axis=1)
        formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
        formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
        df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
        formtemplates=df_melt._categories.apply(pd.Series) \
            .merge(formtemplates, right_index=True, left_index=True) \
            .drop(["_categories"], axis=1) \
            .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
            .drop("variable", axis=1) 
        formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
        formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"id":"_id"},axis=1)
        formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
        formcat=pd.concat([formtemplates2,formtemplates3])
        formcat=formcat.rename({"id":"_id"},axis=1)
        formcat=formcat.loc[:,[
        'name',
        'description',
        'metaKeywords',
        '_categories',
        '_id']]
        formandclone=formandclone.loc[:,[
        '_id',
        '_title',
        '_slug',
        '_description',
        '_language',
        '_featuredCategory',
        'formID',
        'form_type',
        'source',
        'date']]
        formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
        formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
        formclonecategnonnany.index=formclonecategnonnany.date
        formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
        formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
        formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
        formclonecategnonnany["week"]=formclonecategnonnany.index.week
        formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
        formclonecategnonnany["year"]=formclonecategnonnany.index.year
        formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
        formclonecategnonnany["month"]=formclonecategnonnany.index.month
        formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
        df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
        df.index=df.date.tolist()
        df.index=pd.to_datetime(df.index)
        df.index=df.index.to_period("D")
        df.index=df.index.to_timestamp()
        df.index=df.index.strftime('%d-%m-%Y')
        df["date"]=pd.to_datetime(df["date"])
        df=df.rename({"count(_id)":"y"},axis=1)
        df=df.rename({"date":"ds"},axis=1)
        df=df.loc[:,["y","ds"]]
        if cov=="Noncovid":
            df2=df.copy()
        elif cov=="Covid":
            df2=df.loc["01-07-2021":,]
        df2=df2.resample('7D',on="ds").sum()
        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
        season=season.fillna(0)
        season=season.sort_index()
        season.index=season.index.to_period("D")
        season.index=season.index.to_timestamp()
        season.index=season.index.strftime('%d-%m-%Y')
        season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
        fig=make_subplots(rows=4, cols=1,
                        subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
        fig.add_trace(
            go.Scatter(x=season.index,y=season.obs),row=1,col=1
            
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.trend),row=2,col=1
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
            
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.resid),row=4,col=1
            
        )


        fig.update_layout(height=1000, width=950)
        return fig
    elif type(yaxis) is list and len(yaxis)>=2:
        formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis[0]].copy()
        formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
        formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
        formtemplates=formtemplates.rename({"_id":"id"},axis=1)
        formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
        formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
        df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
        formtemplates=df_melt._categories.apply(pd.Series) \
            .merge(formtemplates, right_index=True, left_index=True) \
            .drop(["_categories"], axis=1) \
            .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
            .drop("variable", axis=1) 
        formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
        formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"id":"_id"},axis=1)
        formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
        formcat=pd.concat([formtemplates2,formtemplates3])
        formcat=formcat.rename({"id":"_id"},axis=1)
        formcat=formcat.loc[:,[
        'name',
        'description',
        'metaKeywords',
        '_categories',
        '_id']]
        formandclone=formandclone.loc[:,[
        '_id',
        '_title',
        '_slug',
        '_description',
        '_language',
        '_featuredCategory',
        'formID',
        'form_type',
        'source',
        'date']]
        formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
        formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
        formclonecategnonnany.index=formclonecategnonnany.date
        formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
        formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
        formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
        formclonecategnonnany["week"]=formclonecategnonnany.index.week
        formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
        formclonecategnonnany["year"]=formclonecategnonnany.index.year
        formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
        formclonecategnonnany["month"]=formclonecategnonnany.index.month
        formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
        df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
        df.index=df.date.tolist()
        df.index=pd.to_datetime(df.index)
        df.index=df.index.to_period("D")
        df.index=df.index.to_timestamp()
        df.index=df.index.strftime('%d-%m-%Y')
        df["date"]=pd.to_datetime(df["date"])
        df=df.rename({"count(_id)":"y"},axis=1)
        df=df.rename({"date":"ds"},axis=1)
        df=df.loc[:,["y","ds"]]
        if cov=="Noncovid":
            df2=df.copy()
        elif cov=="Covid":
            df2=df.loc["01-07-2021":,]
        df2=df2.resample('7D',on="ds").sum()
        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
        season=season.fillna(0)
        season=season.sort_index()
        season.index=season.index.to_period("D")
        season.index=season.index.to_timestamp()
        season.index=season.index.strftime('%d-%m-%Y')
        season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
        fig=make_subplots(rows=4, cols=1,
                        subplot_titles=(yaxis[0], "Trend", "Seasonal_yearly","resid"))
        fig.add_trace(
            go.Scatter(x=season.index,y=season.obs),row=1,col=1
            
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.trend),row=2,col=1
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
            
        )
        fig.add_trace(
            go.Scatter(x=season.index,y=season.resid),row=4,col=1
            
        )


        fig.update_layout(height=1000, width=950)
        formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
        formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
        formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
        formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis[1]].copy()
        formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
        formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
        formtemplates=formtemplates.rename({"_id":"id"},axis=1)
        formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
        formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
        df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
        formtemplates=df_melt._categories.apply(pd.Series) \
            .merge(formtemplates, right_index=True, left_index=True) \
            .drop(["_categories"], axis=1) \
            .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
            .drop("variable", axis=1) 
        formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
        formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
        formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
        formtemplates=formtemplates.rename({"id":"_id"},axis=1)
        formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
        formcat=pd.concat([formtemplates2,formtemplates3])
        formcat=formcat.rename({"id":"_id"},axis=1)
        formcat=formcat.loc[:,[
        'name',
        'description',
        'metaKeywords',
        '_categories',
        '_id']]
        formandclone=formandclone.loc[:,[
        '_id',
        '_title',
        '_slug',
        '_description',
        '_language',
        '_featuredCategory',
        'formID',
        'form_type',
        'source',
        'date']]
        formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
        formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
        formclonecategnonnany.index=formclonecategnonnany.date
        formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
        formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
        formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
        formclonecategnonnany["week"]=formclonecategnonnany.index.week
        formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
        formclonecategnonnany["year"]=formclonecategnonnany.index.year
        formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
        formclonecategnonnany["month"]=formclonecategnonnany.index.month
        formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
        df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
        df.index=df.date.tolist()
        df.index=pd.to_datetime(df.index)
        df.index=df.index.to_period("D")
        df.index=df.index.to_timestamp()
        df.index=df.index.strftime('%d-%m-%Y')
        df["date"]=pd.to_datetime(df["date"])
        df=df.rename({"count(_id)":"y"},axis=1)
        df=df.rename({"date":"ds"},axis=1)
        df=df.loc[:,["y","ds"]]
        if cov=="Noncovid":
            df2=df.copy()
        elif cov=="Covid":
            df2=df.loc["01-07-2021":,]
        df2=df2.resample('7D',on="ds").sum()
        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
        season=season.fillna(0)
        season=season.sort_index()
        season.index=season.index.to_period("D")
        season.index=season.index.to_timestamp()
        season.index=season.index.strftime('%d-%m-%Y')
        season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
        fig1=make_subplots(rows=4, cols=1,
                        subplot_titles=(yaxis[1], "Trend", "Seasonal_yearly","resid"))
        fig1.add_trace(
            go.Scatter(x=season.index,y=season.obs),row=1,col=1
            
        )
        fig1.add_trace(
            go.Scatter(x=season.index,y=season.trend),row=2,col=1
        )
        fig1.add_trace(
            go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
            
        )
        fig1.add_trace(
            go.Scatter(x=season.index,y=season.resid),row=4,col=1
            
        )

        fig1.update_layout(height=1000, width=950)
        return fig,fig1
if __name__ == '__main__':
	app.run(host='127.0.0.1', port='8050',debug=True)
    


In [ ]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
import torch 
from sklearn.model_selection import TimeSeriesSplit
from dash import Dash,dcc, html, Input, Output, callback



app = Dash(__name__)
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Tax Forms",
                id="y1",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov1"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Summer Camp Evaluations",
                id="y2",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov2"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic1")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic2")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Virtual Event Forms",
                id="y3",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov3"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Event Feedback Forms",
                id="y4",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov4"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic3")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic4")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
   html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Event Evaluation Forms",
                id="y5",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov5"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Event Planner Forms",
                id="y6",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov6"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic5")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic6")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
   html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Event Registration Forms",
                id="y7",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov7"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Event Booking",
                id="y8",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov8"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic7")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic8")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
   html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Gaming Forms",
                id="y9",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov9"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Summer Camp Consent Forms",
                id="y10",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov10"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic9")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic10")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
        ]
    )

@callback(
    Output("plotindic1","figure"),
    Input("y1","value"),
    Input("cov1","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic2","figure"),
    Input("y2","value"),
    Input("cov2","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic3","figure"),
    Input("y3","value"),
    Input("cov3","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic4","figure"),
    Input("y4","value"),
    Input("cov4","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic5","figure"),
    Input("y5","value"),
    Input("cov5","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic6","figure"),
    Input("y6","value"),
    Input("cov6","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig

@callback(
    Output("plotindic7","figure"),
    Input("y7","value"),
    Input("cov7","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig

@callback(
    Output("plotindic8","figure"),
    Input("y8","value"),
    Input("cov8","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic9","figure"),
    Input("y9","value"),
    Input("cov9","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic10","figure"),
    Input("y10","value"),
    Input("cov10","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
if __name__ == '__main__':
	app.run(host='127.0.0.1', port='8050',debug=True)
    


In [ ]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
import torch 
from sklearn.model_selection import TimeSeriesSplit
from dash import Dash,dcc, html, Input, Output, callback



app = Dash(__name__)
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Tax Forms",
                id="y1",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov1"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Summer Camp Evaluations",
                id="y2",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov2"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic1")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic2")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Virtual Event Forms",
                id="y3",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov3"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Event Feedback Forms",
                id="y4",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov4"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic3")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic4")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
   html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Event Evaluation Forms",
                id="y5",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov5"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Event Planner Forms",
                id="y6",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov6"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic5")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic6")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
   html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Event Registration Forms",
                id="y7",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov7"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Event Booking",
                id="y8",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov8"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic7")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic8")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
   html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Gaming Forms",
                id="y9",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov9"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Summer Camp Consent Forms",
                id="y10",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov10"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic9")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic10")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
        ]
    )

@callback(
    Output("plotindic1","figure"),
    Output("plotindic2","figure"),
    Output("plotindic3","figure"),
    Output("plotindic4","figure"),
    Output("plotindic5","figure"),
    Output("plotindic6","figure"),
    Output("plotindic7","figure"),
    Output("plotindic8","figure"),
    Output("plotindic9","figure"),
    Output("plotindic10","figure"),
    Input("y1","value"),
    Input("cov1","value"),
    Input("y2","value"),
    Input("cov2","value"),
    Input("y3","value"),
    Input("cov3","value"),
    Input("y4","value"),
    Input("cov4","value"),
    Input("y5","value"),
    Input("cov5","value"),
    Input("y6","value"),
    Input("cov6","value"),
    Input("y7","value"),
    Input("cov7","value"),
    Input("y8","value"),
    Input("cov8","value"),
    Input("y9","value"),
    Input("cov9","value"),
    Input("y10","value"),
    Input("cov10","value")
)
def update_graph(yaxis1,cov1,yaxis2,cov2,yaxis3,cov3,yaxis4,cov4,yaxis5,cov5,yaxis6,cov6,yaxis7,cov7,yaxis8,cov8,yaxis9,cov9,yaxis10,cov10):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis1].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov1=="Noncovid":
        df2=df.copy()
    elif cov1=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis1, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis2].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov2=="Noncovid":
        df2=df.copy()
    elif cov2=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig1=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis2, "Trend", "Seasonal_yearly","resid"))
    fig1.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig1.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig1.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig1.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig1.update_layout(height=1000, width=950)
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis3].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov3=="Noncovid":
        df2=df.copy()
    elif cov3=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig2=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis3, "Trend", "Seasonal_yearly","resid"))
    fig2.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig2.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig2.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig2.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig2.update_layout(height=1000, width=950)
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis4].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov4=="Noncovid":
        df2=df.copy()
    elif cov4=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig3=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis4, "Trend", "Seasonal_yearly","resid"))
    fig3.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig3.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig3.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig3.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig3.update_layout(height=1000, width=950)
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis5].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov5=="Noncovid":
        df2=df.copy()
    elif cov5=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig4=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis5, "Trend", "Seasonal_yearly","resid"))
    fig4.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig4.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig4.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig4.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig4.update_layout(height=1000, width=950)
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis6].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov6=="Noncovid":
        df2=df.copy()
    elif cov6=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig5=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis6, "Trend", "Seasonal_yearly","resid"))
    fig5.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig5.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig5.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig5.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig5.update_layout(height=1000, width=950)
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis7].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov7=="Noncovid":
        df2=df.copy()
    elif cov7=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig6=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis7, "Trend", "Seasonal_yearly","resid"))
    fig6.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig6.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig6.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig6.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig6.update_layout(height=1000, width=950)
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis8].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov8=="Noncovid":
        df2=df.copy()
    elif cov8=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig7=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis8, "Trend", "Seasonal_yearly","resid"))
    fig7.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig7.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig7.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig7.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig7.update_layout(height=1000, width=950)
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis9].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov9=="Noncovid":
        df2=df.copy()
    elif cov9=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig8=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis9, "Trend", "Seasonal_yearly","resid"))
    fig8.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig8.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig8.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig8.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig8.update_layout(height=1000, width=950)
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis10].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov10=="Noncovid":
        df2=df.copy()
    elif cov10=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig9=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis10, "Trend", "Seasonal_yearly","resid"))
    fig9.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig9.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig9.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig9.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig9.update_layout(height=1000, width=950)
    return fig,fig1,fig2,fig3,fig4,fig5,fig6,fig7,fig8,fig9
if __name__ == '__main__':
	app.run(host='127.0.0.1', port='8050',debug=True)
    


In [ ]:
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Tax Forms",
                id="y1",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov1"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Summer Camp Evaluations",
                id="y2",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov2"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic1")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic2")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ])
        ]
    )

@callback(
    Output("plotindic1","figure"),
    Input("y1","value"),
    Input("cov1","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic2","figure"),
    Input("y2","value"),
    Input("cov2","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig

In [ ]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose

from dash import Dash,dcc, html, Input, Output, callback



app = Dash(__name__)
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Tax Forms",
                id="y1",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov1"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Summer Camp Evaluations",
                id="y2",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov2"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic1")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic2")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ])
        ]
    )

@callback(
    Output("plotindic1","figure"),
    Input("y1","value"),
    Input("cov1","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic2","figure"),
    Input("y2","value"),
    Input("cov2","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig

if __name__ == '__main__':
	app.run(host='127.0.0.1', port='8050',debug=True)
    


In [ ]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose

from dash import Dash,dcc, html, Input, Output, callback

formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]=="Tax Forms"].copy()
formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
formtemplates=formtemplates.rename({"_id":"id"},axis=1)
formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
formtemplates=df_melt._categories.apply(pd.Series) \
    .merge(formtemplates, right_index=True, left_index=True) \
    .drop(["_categories"], axis=1) \
    .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
    .drop("variable", axis=1) 
formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
formtemplates=formtemplates.rename({"id":"_id"},axis=1)
formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
formcat=pd.concat([formtemplates2,formtemplates3])
formcat=formcat.rename({"id":"_id"},axis=1)
formcat=formcat.loc[:,[
'name',
'description',
'metaKeywords',
'_categories',
'_id']]
formandclone=formandclone.loc[:,[
'_id',
'_title',
'_slug',
'_description',
'_language',
'_featuredCategory',
'formID',
'form_type',
'source',
'date']]
formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
formclonecategnonnany.index=formclonecategnonnany.date
formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
formclonecategnonnany["week"]=formclonecategnonnany.index.week
formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
formclonecategnonnany["year"]=formclonecategnonnany.index.year
formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
formclonecategnonnany["month"]=formclonecategnonnany.index.month
formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
df.index=df.date.tolist()
df.index=pd.to_datetime(df.index)
df.index=df.index.to_period("D")
df.index=df.index.to_timestamp()
df.index=df.index.strftime('%d-%m-%Y')
df["date"]=pd.to_datetime(df["date"])
df=df.rename({"count(_id)":"y"},axis=1)
df=df.rename({"date":"ds"},axis=1)
df=df.loc[:,["y","ds"]]
df2=df.copy()
df2=df2.resample('7D',on="ds").sum()
decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
season=season.fillna(0)
season=season.sort_index()
season.index=season.index.to_period("D")
season.index=season.index.to_timestamp()
season.index=season.index.strftime('%d-%m-%Y')
season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
fig1=make_subplots(rows=4, cols=1,
                subplot_titles=("Tax Forms", "Trend", "Seasonal_yearly","resid"))
fig1.add_trace(
    go.Scatter(x=season.index,y=season.obs),row=1,col=1
    
)
fig1.add_trace(
    go.Scatter(x=season.index,y=season.trend),row=2,col=1
)
fig1.add_trace(
    go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
    
)
fig1.add_trace(
    go.Scatter(x=season.index,y=season.resid),row=4,col=1
    
)


fig1.update_layout(height=1000, width=950)

formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]=="Summer Camp Evaluations"].copy()
formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
formtemplates=formtemplates.rename({"_id":"id"},axis=1)
formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
formtemplates=df_melt._categories.apply(pd.Series) \
    .merge(formtemplates, right_index=True, left_index=True) \
    .drop(["_categories"], axis=1) \
    .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
    .drop("variable", axis=1) 
formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
formtemplates=formtemplates.rename({"id":"_id"},axis=1)
formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
formcat=pd.concat([formtemplates2,formtemplates3])
formcat=formcat.rename({"id":"_id"},axis=1)
formcat=formcat.loc[:,[
'name',
'description',
'metaKeywords',
'_categories',
'_id']]
formandclone=formandclone.loc[:,[
'_id',
'_title',
'_slug',
'_description',
'_language',
'_featuredCategory',
'formID',
'form_type',
'source',
'date']]
formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
formclonecategnonnany.index=formclonecategnonnany.date
formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
formclonecategnonnany["week"]=formclonecategnonnany.index.week
formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
formclonecategnonnany["year"]=formclonecategnonnany.index.year
formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
formclonecategnonnany["month"]=formclonecategnonnany.index.month
formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
df.index=df.date.tolist()
df.index=pd.to_datetime(df.index)
df.index=df.index.to_period("D")
df.index=df.index.to_timestamp()
df.index=df.index.strftime('%d-%m-%Y')
df["date"]=pd.to_datetime(df["date"])
df=df.rename({"count(_id)":"y"},axis=1)
df=df.rename({"date":"ds"},axis=1)
df=df.loc[:,["y","ds"]]
df2=df.copy()
df2=df2.resample('7D',on="ds").sum()
decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
season=season.fillna(0)
season=season.sort_index()
season.index=season.index.to_period("D")
season.index=season.index.to_timestamp()
season.index=season.index.strftime('%d-%m-%Y')
season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
fig2=make_subplots(rows=4, cols=1,
                subplot_titles=("Summer Camp Evaluations", "Trend", "Seasonal_yearly","resid"))
fig2.add_trace(
    go.Scatter(x=season.index,y=season.obs),row=1,col=1
    
)
fig2.add_trace(
    go.Scatter(x=season.index,y=season.trend),row=2,col=1
)
fig2.add_trace(
    go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
    
)
fig2.add_trace(
    go.Scatter(x=season.index,y=season.resid),row=4,col=1
    
)


fig2.update_layout(height=1000, width=950)


app = Dash(__name__)
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Tax Forms",
                id="y1",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov1"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Summer Camp Evaluations",
                id="y2",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov2"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic1")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic2")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ]),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(figure=fig1)],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(figure=fig2)],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ])
        ]
    )

@callback(
    Output("plotindic1","figure"),
    Input("y1","value"),
    Input("cov1","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic2","figure"),
    Input("y2","value"),
    Input("cov2","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig

if __name__ == '__main__':
	app.run(host='127.0.0.1', port='8050',debug=True)
    


In [ ]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose

from dash import Dash,dcc, html, Input, Output, callback
from decompose import decompose


app = Dash(__name__)
formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
dec=decompose(formtemplatesclones,formtemplatescategories,formtemplates)
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Tax Forms",
                id="y1",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov1"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Summer Camp Evaluations",
                id="y2",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov2"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic1")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic2")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ])
        ]
    )

@callback(
    Output("plotindic1","figure"),
    Input("y1","value"),
    Input("cov1","value")
)
def update_graph(yaxis,cov):
    global dec
    season=dec.sd(yaxis,cov)
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic2","figure"),
    Input("y2","value"),
    Input("cov2","value")
)
def update_graph(yaxis,cov):
    global dec
    season=dec.sd(yaxis,cov)
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig

if __name__ == '__main__':
	app.run(host='127.0.0.1', port='8050',debug=True)
    


In [ ]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose

from dash import Dash,dcc, html, Input, Output, callback

formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")

app = Dash(__name__)
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Tax Forms",
                id="y1",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov1"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Summer Camp Evaluations",
                id="y2",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov2"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic1")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic2")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ])
        ]
    )

@callback(
    Output("plotindic1","figure"),
    Input("y1","value"),
    Input("cov1","value")
)
def update_graph(yaxis,cov):
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic2","figure"),
    Input("y2","value"),
    Input("cov2","value")
)
def update_graph(yaxis,cov):
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig

if __name__ == '__main__':
	app.run(host='127.0.0.1', port='8050',debug=True)
    
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose

from dash import Dash,dcc, html, Input, Output, callback



app = Dash(__name__)
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Tax Forms",
                id="y1",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov1"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%",'display':'inline-block'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                formtemplatescategories.metaKeywords.unique().tolist(),
                "Summer Camp Evaluations",
                id="y2",
                searchable=True
        )
        ],style={"width":"50%",'display':'inline-block'}),
        html.Div([
            dcc.Dropdown(
                ["Covid","Noncovid"],
                'Noncovid',
                id="cov2"
        )
        ],style={"width":"50%","float":"right",'display':'inline-block'})
    ],style={"width":"45%","float":"right",'display':'inline-block'}),
    html.Div(
        [
        html.Div(
            
            [dcc.Graph(id="plotindic1")],style={"width":"50%",'display':'inline-block'}
            
        ),
        html.Div(
            [dcc.Graph(id="plotindic2")],style={"width":"50%","float":"right",'display':'inline-block'}

        )
        ])
        ]
    )

@callback(
    Output("plotindic1","figure"),
    Input("y1","value"),
    Input("cov1","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig
@callback(
    Output("plotindic2","figure"),
    Input("y2","value"),
    Input("cov2","value")
)
def update_graph(yaxis,cov):
    formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
    formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
    formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    formtemplatescategoriesy=formtemplatescategories[formtemplatescategories["metaKeywords"]==yaxis].copy()
    formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
    formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
    formtemplates=formtemplates.rename({"_id":"id"},axis=1)
    formtemplates=formtemplates.rename({"_featuredCategory":"_id"},axis=1)
    formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_featuredCategory"},axis=1)
    df_melt = formtemplates.assign(_categories=formtemplates._categories.str.split(","))
    formtemplates=df_melt._categories.apply(pd.Series) \
        .merge(formtemplates, right_index=True, left_index=True) \
        .drop(["_categories"], axis=1) \
        .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
        .drop("variable", axis=1) 
    formtemplates=formtemplates.rename({"_categories":"_id"},axis=1)
    formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates,how="inner",on="_id")
    formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"_id":"_categories"},axis=1)
    formtemplates=formtemplates.rename({"id":"_id"},axis=1)
    formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
    formcat=pd.concat([formtemplates2,formtemplates3])
    formcat=formcat.rename({"id":"_id"},axis=1)
    formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
    formandclone=formandclone.loc[:,[
    '_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]
    formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
    formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
    formclonecategnonnany.index=formclonecategnonnany.date
    formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
    formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
    formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
    formclonecategnonnany["week"]=formclonecategnonnany.index.week
    formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
    formclonecategnonnany["year"]=formclonecategnonnany.index.year
    formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
    formclonecategnonnany["month"]=formclonecategnonnany.index.month
    formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
    df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
    df.index=df.date.tolist()
    df.index=pd.to_datetime(df.index)
    df.index=df.index.to_period("D")
    df.index=df.index.to_timestamp()
    df.index=df.index.strftime('%d-%m-%Y')
    df["date"]=pd.to_datetime(df["date"])
    df=df.rename({"count(_id)":"y"},axis=1)
    df=df.rename({"date":"ds"},axis=1)
    df=df.loc[:,["y","ds"]]
    if cov=="Noncovid":
        df2=df.copy()
    elif cov=="Covid":
        df2=df.loc["01-07-2021":,]
    df2=df2.resample('7D',on="ds").sum()
    decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
    season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
    season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
    season=season.fillna(0)
    season=season.sort_index()
    season.index=season.index.to_period("D")
    season.index=season.index.to_timestamp()
    season.index=season.index.strftime('%d-%m-%Y')
    season.index=pd.to_datetime(season.index,format='%d-%m-%Y')
    fig=make_subplots(rows=4, cols=1,
                    subplot_titles=(yaxis, "Trend", "Seasonal_yearly","resid"))
    fig.add_trace(
        go.Scatter(x=season.index,y=season.obs),row=1,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.trend),row=2,col=1
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
        
    )
    fig.add_trace(
        go.Scatter(x=season.index,y=season.resid),row=4,col=1
        
    )


    fig.update_layout(height=1000, width=950)
    return fig

if __name__ == '__main__':
	app.run(host='127.0.0.1', port='8050',debug=True)
    
